<a href="https://colab.research.google.com/github/ucfilho/raianars_fly/blob/master/RTC_DE_desmonta_IF_new_11_fev_22_2020.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:

from scipy.optimize import differential_evolution
import numpy as np

In [0]:
# Funcoes utilizadas neste codigo
# Area(U,tempAtual,tempInicial,...): constroi matriz de areas trocadores 
#                                    entra duas correntes sendo chamada no loop
# buildMatriz(x,cols): constroi a matriz deltaT e binaria (logica de trocas)
#
# Temperatura(matrizDeltaT,...): constroi as temperaturas e cargas termicas

In [0]:
def buildMatriz(x,cols):

  nref=int(len(x)/2) # metade de x é deltaT e a outra metade variavel logica
  matrizDeltaT=np.zeros((cols,cols))
  matrizBinaria=np.zeros((cols,cols))

  contador=0
  for i in range(cols):
    for j in range(cols):
      print('%d %d %d %d'%(i,j,contador,nref))
      if(i<j):
        
        matrizDeltaT[i,j]=x[contador]
        matrizBinaria[i,j]=round(x[contador+nref])
        contador=contador+1
        
  return matrizDeltaT,matrizBinaria

In [0]:
def Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial):

  cols=len(mCp)
  cargaTermica=np.zeros((cols,cols))
  contador=0
  for i in range(cols):
    for j in range(cols):
      contador=contador+1
      cargaTermica[i,j]=mCp[i]*matrizDeltaT[i,j]

      if(i<j):
        if(matrizBinaria[i,j]==1):
          if(matrizDeltaT[i,j]<0):#a corrente i perde calor para a j
            #i: CQ, j: CF
            tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
            tempSaidaF=-cargaTermica[i,j]/(mCp[j])+tempAtual[j]
            flag=0 # flag : indica que nao entra no teste logico
            # usei flag no lugar q ifs pq a identacao ficou melhor assim
            if(tempAtual[i] > tempAtual[j]):
              if(tempSaidaQ > tempAtual[j]):
                if(tempAtual[i] > tempSaidaF):
                  if(contador < cols):
                    flag=1
          
            if(flag>0):
              tempAtual[i]=tempSaidaQ
              tempAtual[j]=tempSaidaF

          else: #if(matrizDeltaT[i,j]<0)
            #a corrente i ganha calor da j
            #i: CF, j: CQ
            tempSaidaF=tempAtual[i]+matrizDeltaT[i,j]
            tempSaidaQ=-cargaTermica[i,j]/(mCp[j])+tempAtual[j]
            flag=0

            if(tempAtual[i] > tempAtual[j]):
              if(tempSaidaQ >tempAtual[j]):
                if(tempAtual[i] > tempSaidaF):
                  if(contador < cols):
                    flag=1
           
            if(flag>0): # satisfaz as condicoes do if acima
              tempAtual[i]=tempSaidaF
              tempAtual[j]=tempSaidaQ

  return tempAtual,cargaTermica

In [0]:
def Area(U,tempAtual,tempInicial,cargaTermica,matrizDeltaT):
  cols=len(tempInicial)
  Trocador=np.zeros((cols,cols)) # matriz q armazena as areas dos trocadores
  Soma_Area=0

  for i in range(cols):
    for j in range(cols):
      if(i<j):
        Carga=cargaTermica[i,j]
        # calcula a area de placas das duas correntes que trocam i e j

        if(matrizDeltaT[i,j]<0):
          #a corrente i perde calor para a j
          #i: CQ, j:CF
          tempSaidaF=tempAtual[j]
          tempSaidaQ=tempAtual[i]
          try:
            A=((tempSaidaQ-tempSaidaF)-(tempAtual[i]-tempAtual[j]))
            B=np.log((tempSaidaQ-tempSaidaF)/(tempAtual[i]-tempAtual[j]))
            deltaTMediaLog=A/B
          except:
            deltaTMediaLog=1e99
        else: # if(matrizDeltaT[i,j]<0):
          #a corrente i ganha calor da j
          #i: CF, j: CQ
          tempSaidaF=tempAtual[i]
          tempSaidaQ=tempAtual[j]
          try:
            A=((tempSaidaQ-tempSaidaF)-(tempAtual[j]-tempAtual[i]))
            B=np.log((tempSaidaQ-tempSaidaF)/(tempAtual[j]-tempAtual[i]))
            deltaTMediaLog=A/B
          except:
            deltaTMediaLog=1e99
       
        # parte dentro do if(i<j) que usa o calculo e gera matriz area
        
        if(deltaTMediaLog==1e99):  
          AreaCalc=1e99
        else:
          AreaCalc=abs(Carga/(U*deltaTMediaLog))

        if(tempAtual[i]==tempInicial[i]):
          AreaCalc=0

        if(tempAtual[j]==tempInicial[j]):
          AreaCalc=0

        Soma_Area=Soma_Area+AreaCalc

        Trocador[i,j]=AreaCalc

  return Soma_Area,Trocador

In [0]:
def Utilidades(tempAlvo,tempAtual,mCp):

  cols=len(mCp)
  cargaUQ=0
  cargaUF=0
  areaTrocador=0
  Trocadores=np.zeros(cols)

  for i in range(cols):
    deltaT=tempAlvo[i]-tempAtual[i]
    if(deltaT>0):#usa UQ
      cargaUQ=cargaUQ+mCp[i]*deltaT
      #UQ -> vapor saturado -> entra e sai a 100ºC
      #A=(373.15-(tempAlvo[i])-(373.15-tempAtual[i]))
      #B=np.log((373.15-tempAlvo[i])/(373.15-tempAtual[i]))
      # deltaTMediaLog=A/B
      try:
        A=(373.15-(tempAlvo[i])-(373.15-tempAtual[i]))
        B=np.log((373.15-tempAlvo[i])/(373.15-tempAtual[i]))
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
      areaTrocador=areaTrocador+abs(mCp[i]*(deltaT)/(U*deltaTMediaLog))
    elif(deltaT==0):#nao usa UF e UQ
      deltaTMediaLog=0      
    else:#usa UF      
      cargaUF=cargaUF+mCp[i]*(-deltaT)
      #UF -> água resfriada -> entra a 5ºC e sai a 20ºC
      try:
        A=((tempAlvo[i]-293.15)-(tempAtual[i]-278.15))
        B=np.log((tempAlvo[i]-293.15)/(tempAtual[i]-278.15))
        deltaTMediaLog=A/B
      except:
        deltaTMediaLog=1e99
      
      if(deltaTMediaLog==1e99):
        areaTrocador=1e99
        Trocadores[i]=1e99
      else:
        Trocadores[i]=abs(mCp[i]*(-deltaT)/(U*deltaTMediaLog))
        areaTrocador=areaTrocador+Trocadores[i]

  return areaTrocador, Trocadores, cargaUF,cargaUQ 
  # areaTrocador contem a soma das areas
  # Trocadores a area associada a cada corrente
  # cargaUF e cargaUQ kW gasto

In [0]:
#FUN calcula o TAC para um determinado arranjo
#  cada calculo é feito em uma funcao a parte Fun usa todas funcoes 

def FUN(x):

  global mCp,tempAtual,tempAlvo,U,coefCustoUQ,coefCustoUF
  
  cols=len(mCp)
  tempInicial=np.copy(tempAtual) 
  matrizDeltaT,matrizBinaria=buildMatriz(x,cols) # transforma x em matriz logica e troca
  #print('matrizDeltaT')
  #print(matrizDeltaT)
  tempAtual,Carga=Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial)
  areaTrocador,matrizArea =Area(U,tempAtual,tempInicial,Carga,matrizDeltaT)
  areaUtilidade, TrocaUtilidade, cargaUF, cargaUQ = Utilidades(tempAlvo,tempAtual,mCp)

  costTrocador=300*(areaTrocador+areaUtilidade)**0.5

  costUQ=coefCustoUQ*cargaUQ
  costUF=coefCustoUF*cargaUF

  #print('costUQ');print(costUQ)
  #print('costUF');print(costUF)
  #print('costTrocador');print(costTrocador)
  #print('areaTrocador');print(areaTrocador)
  #print('costUF');print(costUF)
  #print('costUQ');print(costUQ)

  cost=costUQ+costUF+costTrocador

  funr = np.where(np.isnan(cost), 1e99, cost)


  return funr

In [0]:
#MATGRAF retorna o gráfico das trocas térmicas em cada trocador
# function matrizGrafico=MATGRAF(matrizDeltaT,mCp,tempAtual)
def MATGRAF(matrizDeltaT,mCp,tempAtual):
  cols=len(mCp)
  matrizGrafico=np.zeros((cols,cols))
  contador=0
  matrizGrafico[:,0]=tempAtual
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        if(matrizDeltaT[i,j]<0):#a corrente i perde calor para a j
          #i: CQ, j:CF
          cargaTermica=mCp[i]*matrizDeltaT[i,j]
          tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
          tempSaidaF=-cargaTermica[i,j]/(mCp[j])+tempAtual[j]
          if(
              tempAtual[i]>tempAtual[j] and
              tempSaidaQ>tempAtual[j] and
              tempAtual[i]>tempSaidaF and
              contador < cols
              
            ):
            tempAtual[i]=tempSaidaQ
            tempAtual[j]=tempSaidaF
            matrizGrafico[:,contador]=tempAtual
            
            #TQentra>TFentra & TQsai>TFentra & TQentra>TFsai
            contador=contador+1
            # troquei o elif por else (fica melhor no contexto?)
            # elif (matrizDeltaT[i,j]>0) : #a corrente i ganha calor da j
      else : #a corrente i ganha calor da j

        # situa
        cargaTermica=mCp[i]*matrizDeltaT[i,j]
        tempSaidaF=tempAtual[i]+matrizDeltaT[i,j]
        tempSaidaQ=-cargaTermica/(mCp[j])+tempAtual[j] 
        if (
          
            tempAtual[i]<tempAtual[j] and 
            tempSaidaF<tempAtual[j] and 
            tempAtual[i]<tempSaidaQ and
            contador< cols

            ):
            #  i: CF, j: CQ
              
            tempAtual[i]=tempSaidaF
            tempAtual[j]=tempSaidaQ
            matrizGrafico[:,contador]=tempAtual
            contador=contador+1          

  return matrizGrafico

In [0]:
#MATBEST retorna a matriz com os deltas das trocas térmicas que ocorreram e a matriz binária
# function [matrizTrocaBest,matrizBinaria]=MATBEST(x,mCp,tempAtual)
def MATBEST(x,mCp,tempAtual):
  
  nref=int(len(x)/2)
  #nref=0
  cols=len(mCp)

  # cada particula contem metade das variaveis para matriz binaria
  # e outra metade para deltaT nref e para separar os dois casos
  matrizDeltaT=np.zeros((cols,cols))
  matrizTrocaBest=np.zeros((cols,cols))
  matrizBinaria=np.zeros((cols,cols))

  # contador=-1
  contador=0
  for i in range(cols):
    for j in range(cols):
      if(i<j):
        # contador=contador+1
        matrizDeltaT[i,j]=x[contador]
        matrizBinaria[i,j]=round(x[contador+nref])
        contador=contador+1
    
  
    tempSaidaQ=0;tempSaidaF=0;cargaTermica=0
    for i in range(cols):
      for j in range(cols):
        if(i<j):
          if(matrizBinaria[i,j]==1):
            if(matrizDeltaT[i,j]<0):#a corrente i perde calor para a j 
              #i: CQ, j: CF
              cargaTermica=mCp[i]*matrizDeltaT[i,j]
              tempSaidaQ=tempAtual[i]+matrizDeltaT[i,j]
              tempSaidaF=-cargaTermica/(mCp[j])+tempAtual[j]
              if(
                  tempAtual[i]>tempAtual[j] and
                  tempSaidaQ>tempAtual[j] and
                  tempAtual[i]>tempSaidaF
                ): #TQentra>TFentra & TQsai>TFentra & TQentra>TFsai
                tempAtual[i]=tempSaidaQ
                tempAtual[j]=tempSaidaF
              else:
                matrizDeltaT[i,j]=0
            else:#a corrente i ganha calor da j
              # i: CF, j:CQ
              cargaTermica=mCp[i]*matrizDeltaT[i,j]
              tempSaidaF=tempAtual[i]+matrizDeltaT[i,j]
              tempSaidaQ=-cargaTermica/(mCp[j])+tempAtual[j]
              if(
                  tempAtual[i]<tempAtual[j] and
                  tempSaidaF<tempAtual[j] and
                  tempAtual[i]<tempSaidaQ
                 ):
                #TFentra<TQentra & TFsai<TQentra & TFentra<TQsai
                tempAtual[i]=tempSaidaF
                tempAtual[j]=tempSaidaQ
              else:
                matrizDeltaT[i,j]=0
          else:
            matrizDeltaT[i,j]=0
  #print('contador=',contador)
  matrizTrocaBest=matrizDeltaT
  return matrizTrocaBest,matrizBinaria

In [0]:
#********************PROGRAMA PRINCIPAL*************************

NPAR=100 #Número de partículas
ITE=500 #Número de iterações
PAR=12 #Número de parâmetros a ser otimizados

MAX=[100,100,100,100,100,100,1,1,1,1,1,1]
MIN=[-100,-100,-100,-100,-100,-100,0,0,0,0,0,0]

mCp=[4,2,1.5,3] #[kW/K]
tempInicial=[453,393,523,533] #[K]
tempAtual=[453,393,523,533] #[K]
tempAlvo=[513,508,403,433] #[K]
U=0.2 #[kW/(m2.K)]
coefCustoUQ=110 #[$/(kW.ano)]
coefCustoUF=12.2 #[$/(kW.ano)]

bounds = []
for i in range(12):
  bounds.append((MAX[i],MIN[i]))



In [14]:

fchoice=FUN
result = differential_evolution(fchoice, bounds, maxiter=ITE, popsize=NPAR)
print('temperatura atual=',tempAtual)
print(result.x, result.fun)
print(tempAtual)
GBEST=result.x
BEST=result.fun
matrizTrocaBest,matrizBinaria=MATBEST(GBEST,mCp,tempAtual)
matrizGrafico=MATGRAF(matrizTrocaBest,mCp,tempAtual)

print('Melhor solução para %d iterações com %d partículas \n'%(ITE,NPAR));print(GBEST)
print('Valor da função f(gbest)=%f \n'%BEST)
print('matriz grafico');print(matrizGrafico)
#print('Matriz de Trocas Térmicas \n')
#print(matrizTrocaBest)
#print('Matriz Binária \n')
#print(matrizBinaria)


0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: RuntimeWarning: invalid value encountered in double_scalars
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:20: RuntimeWarning: invalid value encountered in double_scalars


Streaming output truncated to the last 5000 lines.
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
1 0 3 6
1 1 3 6
1 2 3 6
1 3 4 6
2 0 5 6
2 1 5 6
2 2 5 6
2 3 5 6
3 0 6 6
3 1 6 6
3 2 6 6
3 3 6 6
0 0 0

In [15]:
cols=int(len(GBEST)/2)
matrizDeltaT,matrizBinaria=buildMatriz(GBEST,cols) # transforma x em matriz logica e troca
tempAtual,Carga=Temperatura_Carga(mCp,matrizDeltaT,matrizBinaria,tempInicial)
Soma_Area,Trocador=Area(U,tempAtual,tempInicial,cargaTermica,matrizDeltaT)
print('temperatura atual');print(tempAtual)
print('temperatura inicial');print(tempInicial)
print('trocador');print(Trocador)

0 0 0 6
0 1 0 6
0 2 1 6
0 3 2 6
0 4 3 6
0 5 4 6
1 0 5 6
1 1 5 6
1 2 5 6
1 3 6 6


IndexError: ignored

In [0]:
print(cols)
print(GBEST)